In [67]:
# Obtaining the text from an URL
import urllib.request

url = "http://research.ics.aalto.fi/cog/data/udhr/txt/cln.txt"

with urllib.request.urlopen(url) as response:
    text = response.read().decode('utf-8') # Read document and decode it with UTF-8

print(len(text))

11176


### Base for the tokenizers

In [68]:
def get_stats(ids, counts=None): # It takes a list of integers and returns a dictionary of counts of consecutive paris.
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx): # In the list of integers (ids), it replaces the consecutive occurrences of paris with the new integer token idx.
    newids = []
    i = 0
    while i < len(ids):
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids


# Base class for the tokenizers

class Tokenizer:

    def __init__(self): # By default the vocab size is 256 (all bytes), without merges and patterns.
        self.merges = {} # (int, int) -> int
        self.pattern = "" # str
        self.special_tokens = {} # str -> int
        self.vocab = self._build_vocab() # int -> bytes

    def _build_vocab(self): # Building the vocabulary from the merges (deterministically derived).
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab


### Basic tokenizer

In [69]:
## Creating the class for the basic tokenizer, taking as a base the class Tokenizer that we have created before.

class BasicTokenizer(Tokenizer):

    def __init__(self):
        super().__init__()

    def train(self, text, vocab_size, verbose=False): # Creating the training step.
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # Input text for preprocessing.
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers

        # Iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes
        for i in range(num_merges):
            # count the number of times every consecutive pair appears
            stats = get_stats(ids)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = merge(ids, pair, idx)
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # print it
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # Save class variables for later using/printing them.
        self.merges = merges
        self.vocab = vocab

In [70]:
# Training the tokenizer with the new class BasicTokenizer.
basic_tokenizer = BasicTokenizer()
training_text = text
vocab_size = 456  # Vocabulary size (256 bytes + 200 merges)
basic_tokenizer.train(training_text, vocab_size)

In [71]:
# Visualising the results (subwords) obtained from the first 20 merges.
for index, (k, v) in enumerate(basic_tokenizer.vocab.items()):
    if 256 <= index < 276:
        print(f"{k} => {v}")

256 => b'a '
257 => b's '
258 => b'e '
259 => b'en'
260 => b'er'
261 => b' d'
262 => b'ci'
263 => b're'
264 => b'on'
265 => b'al'
266 => b'i '
267 => b't '
268 => b', '
269 => b'ta'
270 => b'el'
271 => b'. '
272 => b'la '
273 => b'\xc3\xb3'
274 => b'ti'
275 => b'per'


### Regex tokenizer (adapted)

In [72]:
## Creating the class for the regex tokenizer, taking as a base the class Tokenizer that we have created before.

import regex as re

# Pattern for text spliting, based on the GPT4 pattern, but adapted to Catalan.
GPT4_SPLIT_PATTERN = r"""'(?i:[\w]+|'l'|'ls|'m'|'t'|'ns|-l.+|n.+|s.+|l'|d'|n'|s'|t')|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class RegexTokenizer(Tokenizer):

    def __init__(self, pattern=None):
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern # Here we say we want to use our specific pattern.
        self.compiled_pattern = re.compile(self.pattern)
        self.special_tokens = {}
        self.inverse_special_tokens = {}

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        text_chunks = re.findall(self.compiled_pattern, text) # Split the text up into text chunks, using the pattern created before.

        ids = [list(ch.encode("utf-8")) for ch in text_chunks] # Input text preprocessing

        # Iteratively merge the most common pairs to create the new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
        for i in range(num_merges):
            # Count the number of times that appears every consecutive pair.
            stats = {}
            for chunk_ids in ids:
                get_stats(chunk_ids, stats)
            pair = max(stats, key=stats.get) # find the pair with the highest count
            idx = 256 + i # assigning to the new token the next available id.
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            merges[pair] = idx # save the merge
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # Save the class variables for later using/printing them.
        self.merges = merges
        self.vocab = vocab

    def register_special_tokens(self, special_tokens):
        # Special_tokens is a dictionary of str -> int
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}


In [73]:
# Training the tokenizer with the new class RegexTokenizer.
regex_tokenizer =  RegexTokenizer()
training_text = text
vocab_size = 456 # Vocabulary size (256 bytes + 200 merges)
regex_tokenizer.train(training_text, vocab_size)

In [74]:
# Visualising the results (subwords) obtained from the first 20 merges.
for index, (k, v) in enumerate(regex_tokenizer.vocab.items()):
    if 256 <= index < 276:
        print(f"{k} => {v}")

256 => b' d'
257 => b'en'
258 => b' l'
259 => b'es'
260 => b'er'
261 => b' a'
262 => b' p'
263 => b'ci'
264 => b' i'
265 => b'ta'
266 => b' de'
267 => b'on'
268 => b're'
269 => b' s'
270 => b'al'
271 => b' la'
272 => b' c'
273 => b'\xc3\xb3'
274 => b'ti'
275 => b' e'
